Importation of libraries

In [414]:
%pylab inline
import pandas as pd
import numpy as np
from collections import Counter
from copy import copy
from string import ascii_lowercase
from string import ascii_uppercase

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


An example of how the order of preferences is defined for men and women

In [409]:
man_list = ['a', 'b', 'c', 'd']
women_list = ['A', 'B', 'C', 'D']

In [410]:
man_df = pd.DataFrame({'A': [1,3,2,1], 'B': [2,1,3,2], 'C':[3,2,1,3], 'D':[4,4,4,4]})
man_df.index = man_list

In [411]:
women_df = pd.DataFrame({'A': [4,1,2,3], 'B': [3,4,1,2], 'C':[2,3,4,1], 'D':[1,2,3,4]})
women_df.index = man_list

Definition of functions: 
1) Functions that create the lists for the men and the women. Below each definition is an example of how they appear.

In [491]:
def createManLists (n):
  man_list=[]
  for x in range (0,n):
        man_list.append('M'+str(x+1))
  return man_list

In [492]:
print(createManLists(4))

['M1', 'M2', 'M3', 'M4']


In [493]:
def createWomanLists (n):
  women_list=[]
  for x in range (0,n):
        women_list.append('W'+str(x+1))
  return women_list

In [494]:
print(createWomanLists(4))

['W1', 'W2', 'W3', 'W4']


2)Definition of functions creates the lists of preferences.
  First are the functions that create the preferences for the worst case     possible. The women follow have the following order: 
  (n, n-1, n-2 …, 1

  1, n, n-1, …, 2

  2,1,n,n-1,…,3

  ... … …

   n-1,n-2,…,1,n)

   And the men the next: 

   (1,2,3,4,…,n


   n-1,1,2,3,…,n

   n-2,n-1,1,2,…,n

   … … … 

   1,2,3,4,..,n)



In [463]:
def CreateWomenProposals(n):
  dict={}
  initial=n
  for k in range(1,n+1):
    j=initial
    prop_array=[]
    for i in range(1,n+1):
      #dict[i]=j
      prop_array.append(j)
      j+=1
      if j==n+1:
        j=1
      #j-=1
    #print(prop_array)
    dict['W'+str(k)]=prop_array
    initial-=1

  return dict

In [464]:
print(CreateWomenProposals(4))

{'W1': [4, 1, 2, 3], 'W2': [3, 4, 1, 2], 'W3': [2, 3, 4, 1], 'W4': [1, 2, 3, 4]}


In [476]:
def CreateMenProposals(n):
  dict={}
  initial=n
  matriz=np.zeros((n, n))
  a=1
  for i in range(0,n):
    if a==0:
      a=n-1
    k=a
    #prop_array=[]
    for j in range(0,n):
      #dict[i]=j
      #print(j)
      if j==n-1:
        matriz[i][j]=n
      else:
        matriz[i][j]=k
      k+=1
      if k==n:
        k=1
      #j-=1
    #print(prop_array)
    a-=1
  #print(matriz)
  matriz=matriz.astype(int)
  for each in range (1,n+1):
    prop_array=[]
    for j in range(0,n):
      prop_array.append(matriz[j][each-1])
    dict['W'+str(each)]=prop_array
  return dict

In [477]:
print(CreateMenProposals(4))

{'W1': [1, 3, 2, 1], 'W2': [2, 1, 3, 2], 'W3': [3, 2, 1, 3], 'W4': [4, 4, 4, 4]}


We define the tables that organize the arrengement preferences for n=4:

In [716]:
women_df = pd.DataFrame(CreateWomenProposals(4))
women_df.index = createManLists(4)

In [717]:
women_df

,W1,W2,W3,W4
M1,4,3,2,1
M2,1,4,3,2
M3,2,1,4,3
M4,3,2,1,4


In [718]:
man_df = pd.DataFrame(CreateMenProposals(4))
man_df.index = createManLists(4)

In [719]:
man_df

,W1,W2,W3,W4
M1,1,2,3,4
M2,3,1,2,4
M3,2,3,1,4
M4,1,2,3,4


Here we execute the algorithm that finds the best possible arrengements.
We define the algorithm that at the end prints the gotten arrengements and the number of times the algorithm was executed.

In [691]:
def GaleShapely_Algorithm (man_list,women_list,women_available):
  count=0
  waiting_list = []
  proposals = {}
  # while not all men have pairs
  while len(waiting_list)<len(man_list):
      # man makes proposals
      for man in man_list:
          if man not in waiting_list:
              # each man make proposal to the top women from it's list
              women = women_available[man]
              #print(man_df.loc[man][man_df.loc[man].index.isin(women)].idxmin())
              best_choice = man_df.loc[man][man_df.loc[man].index.isin(women)].idxmin()
              proposals[(man, best_choice)]=(man_df.loc[man][best_choice],women_df.loc[man][best_choice])
      # if women have more than one proposals 
      # she will choose the best option
      overlays = Counter([key[1] for key in proposals.keys()])
      # cycle to choose the best options
      for women in overlays.keys():
          if overlays[women]>1:
              # pairs to drop from proposals
              pairs_to_drop = sorted({pair: proposals[pair] for pair in proposals.keys() 
                      if women in pair}.items(), 
                    key=lambda x: x[1][1]
                    )[1:]
              # if man was rejected by woman
              # there is no pint for him to make proposal 
              # second time to the same woman
              for p_to_drop in pairs_to_drop:
                  del proposals[p_to_drop[0]]
                  _women = copy(women_available[p_to_drop[0][0]])
                  _women.remove(p_to_drop[0][1])
                  women_available[p_to_drop[0][0]] = _women
      # man who successfully created pairs must be added to the waiting list 
      waiting_list = [man[0] for man in proposals.keys()]
      # update counter
      count+=1
  print("Proposals: ")
  print(proposals)
  print("Times executed: ")
  print(count)

In [720]:
GaleShapely_Algorithm(createManLists(4),createWomanLists(4),{man:women_list for man in man_list})

Proposals: 
{('M2', 'W1'): (3, 1), ('M3', 'W2'): (3, 1), ('M4', 'W3'): (3, 1), ('M1', 'W4'): (4, 1)}
Times executed: 
10


Now we will do the same previous procedure with n= 40.

In [721]:
women_df = pd.DataFrame(CreateWomenProposals(40))
women_df.index = createManLists(40)

In [722]:
women_df

,W1,W2,W3,W4,W5,W6,W7,W8,W9,W10,W11,W12,W13,W14,W15,W16,W17,W18,W19,W20,W21,W22,W23,W24,W25,W26,W27,W28,W29,W30,W31,W32,W33,W34,W35,W36,W37,W38,W39,W40
M1,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1
M2,1,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2
M3,2,1,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3
M4,3,2,1,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4
M5,4,3,2,1,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5
M6,5,4,3,2,1,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6
M7,6,5,4,3,2,1,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7
M8,7,6,5,4,3,2,1,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8
M9,8,7,6,5,4,3,2,1,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9
M10,9,8,7,6,5,4,3,2,1,40,39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10


In [723]:
man_df = pd.DataFrame(CreateMenProposals(40))
man_df.index = createManLists(40)

In [724]:
man_df

,W1,W2,W3,W4,W5,W6,W7,W8,W9,W10,W11,W12,W13,W14,W15,W16,W17,W18,W19,W20,W21,W22,W23,W24,W25,W26,W27,W28,W29,W30,W31,W32,W33,W34,W35,W36,W37,W38,W39,W40
M1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
M2,39,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,40
M3,38,39,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,40
M4,37,38,39,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,40
M5,36,37,38,39,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,40
M6,35,36,37,38,39,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,40
M7,34,35,36,37,38,39,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,40
M8,33,34,35,36,37,38,39,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,40
M9,32,33,34,35,36,37,38,39,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,40
M10,31,32,33,34,35,36,37,38,39,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,40


In [726]:
GaleShapely_Algorithm(createManLists(40),createWomanLists(40),{man:women_list for man in man_list})

Proposals: 
{('M2', 'W1'): (39, 1), ('M3', 'W2'): (39, 1), ('M4', 'W3'): (39, 1), ('M5', 'W4'): (39, 1), ('M6', 'W5'): (39, 1), ('M7', 'W6'): (39, 1), ('M8', 'W7'): (39, 1), ('M9', 'W8'): (39, 1), ('M10', 'W9'): (39, 1), ('M11', 'W10'): (39, 1), ('M12', 'W11'): (39, 1), ('M13', 'W12'): (39, 1), ('M14', 'W13'): (39, 1), ('M15', 'W14'): (39, 1), ('M16', 'W15'): (39, 1), ('M17', 'W16'): (39, 1), ('M18', 'W17'): (39, 1), ('M19', 'W18'): (39, 1), ('M20', 'W19'): (39, 1), ('M21', 'W20'): (39, 1), ('M22', 'W21'): (39, 1), ('M23', 'W22'): (39, 1), ('M24', 'W23'): (39, 1), ('M25', 'W24'): (39, 1), ('M26', 'W25'): (39, 1), ('M27', 'W26'): (39, 1), ('M28', 'W27'): (39, 1), ('M29', 'W28'): (39, 1), ('M30', 'W29'): (39, 1), ('M31', 'W30'): (39, 1), ('M32', 'W31'): (39, 1), ('M33', 'W32'): (39, 1), ('M34', 'W33'): (39, 1), ('M35', 'W34'): (39, 1), ('M36', 'W35'): (39, 1), ('M37', 'W36'): (39, 1), ('M38', 'W37'): (39, 1), ('M39', 'W38'): (39, 1), ('M40', 'W39'): (39, 1), ('M1', 'W40'): (40, 1)}
Times

Now we execute the algorithm for the best possible case.  This case is the one where each man has as maximum preference a woman that also has him as the maximum preference.

In [581]:
def CreateWomenProposalsBest(n):
  dict={}
  #initial=1
  count=1
  for k in range(1,n+1):
    j=2
    prop_array=[]
    for i in range(1,n+1):
      #dict[i]=j
      if count==i:
        prop_array.append(1)
      else:
        prop_array.append(j)
        j+=1
      #if j==n+1:
      #  j=1
      #j-=1
    #print(prop_array)
    dict['W'+str(k)]=prop_array
    #initial+=1
    count+=1

  return dict

In [582]:
print(CreateWomenProposalsBest(4))

{'W1': [1, 2, 3, 4], 'W2': [2, 1, 3, 4], 'W3': [2, 3, 1, 4], 'W4': [2, 3, 4, 1]}


In [658]:
def CreateMenProposalsBest(n):
  dict={}
  initial=n
  matriz=np.zeros((n, n))
  for i in range(0,n):
    a=2
    for j in range(0,n):
      if i==j:
        matriz[i][j]=1
      else:
        matriz[i][j]=a
        a+=1

  #print(matriz)
  matriz=matriz.astype(int)
  for each in range (1,n+1):
    prop_array=[]
    for j in range(0,n):
      prop_array.append(matriz[j][each-1])
    dict['W'+str(each)]=prop_array
  return dict

In [659]:
print(CreateMenProposalsBest(4))

{'W1': [1, 2, 2, 2], 'W2': [2, 1, 3, 3], 'W3': [3, 3, 1, 4], 'W4': [4, 4, 4, 1]}


In [727]:
women_df = pd.DataFrame(CreateWomenProposalsBest(4))
women_df.index = createManLists(4)

In [728]:
man_df = pd.DataFrame(CreateMenProposalsBest(4))
man_df.index = createManLists(4)

These are the best possible marriage proposals:

In [729]:
women_df

,W1,W2,W3,W4
M1,1,2,2,2
M2,2,1,3,3
M3,3,3,1,4
M4,4,4,4,1


In [730]:
man_df

,W1,W2,W3,W4
M1,1,2,3,4
M2,2,1,3,4
M3,2,3,1,4
M4,2,3,4,1


In [731]:
GaleShapely_Algorithm(createManLists(4),createWomanLists(4),{man:women_list for man in man_list})

Proposals: 
{('M1', 'W1'): (1, 1), ('M2', 'W2'): (1, 1), ('M3', 'W3'): (1, 1), ('M4', 'W4'): (1, 1)}
Times executed: 
1


Because each person were able to marry their highest preference, the algorithm was only executed once. 

Now we will do the same procedure for 40 couples:

In [732]:
women_df = pd.DataFrame(CreateWomenProposalsBest(40))
women_df.index = createManLists(40)

In [733]:
man_df = pd.DataFrame(CreateMenProposalsBest(40))
man_df.index = createManLists(40)

In [734]:
women_df

,W1,W2,W3,W4,W5,W6,W7,W8,W9,W10,W11,W12,W13,W14,W15,W16,W17,W18,W19,W20,W21,W22,W23,W24,W25,W26,W27,W28,W29,W30,W31,W32,W33,W34,W35,W36,W37,W38,W39,W40
M1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
M2,2,1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
M3,3,3,1,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
M4,4,4,4,1,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
M5,5,5,5,5,1,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
M6,6,6,6,6,6,1,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
M7,7,7,7,7,7,7,1,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8
M8,8,8,8,8,8,8,8,1,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
M9,9,9,9,9,9,9,9,9,1,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
M10,10,10,10,10,10,10,10,10,10,1,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11


In [735]:
man_df

,W1,W2,W3,W4,W5,W6,W7,W8,W9,W10,W11,W12,W13,W14,W15,W16,W17,W18,W19,W20,W21,W22,W23,W24,W25,W26,W27,W28,W29,W30,W31,W32,W33,W34,W35,W36,W37,W38,W39,W40
M1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
M2,2,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
M3,2,3,1,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
M4,2,3,4,1,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
M5,2,3,4,5,1,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
M6,2,3,4,5,6,1,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
M7,2,3,4,5,6,7,1,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
M8,2,3,4,5,6,7,8,1,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
M9,2,3,4,5,6,7,8,9,1,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
M10,2,3,4,5,6,7,8,9,10,1,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40


In [736]:
GaleShapely_Algorithm(createManLists(40),createWomanLists(40),{man:women_list for man in man_list})

Proposals: 
{('M1', 'W1'): (1, 1), ('M2', 'W2'): (1, 1), ('M3', 'W3'): (1, 1), ('M4', 'W4'): (1, 1), ('M5', 'W5'): (1, 1), ('M6', 'W6'): (1, 1), ('M7', 'W7'): (1, 1), ('M8', 'W8'): (1, 1), ('M9', 'W9'): (1, 1), ('M10', 'W10'): (1, 1), ('M11', 'W11'): (1, 1), ('M12', 'W12'): (1, 1), ('M13', 'W13'): (1, 1), ('M14', 'W14'): (1, 1), ('M15', 'W15'): (1, 1), ('M16', 'W16'): (1, 1), ('M17', 'W17'): (1, 1), ('M18', 'W18'): (1, 1), ('M19', 'W19'): (1, 1), ('M20', 'W20'): (1, 1), ('M21', 'W21'): (1, 1), ('M22', 'W22'): (1, 1), ('M23', 'W23'): (1, 1), ('M24', 'W24'): (1, 1), ('M25', 'W25'): (1, 1), ('M26', 'W26'): (1, 1), ('M27', 'W27'): (1, 1), ('M28', 'W28'): (1, 1), ('M29', 'W29'): (1, 1), ('M30', 'W30'): (1, 1), ('M31', 'W31'): (1, 1), ('M32', 'W32'): (1, 1), ('M33', 'W33'): (1, 1), ('M34', 'W34'): (1, 1), ('M35', 'W35'): (1, 1), ('M36', 'W36'): (1, 1), ('M37', 'W37'): (1, 1), ('M38', 'W38'): (1, 1), ('M39', 'W39'): (1, 1), ('M40', 'W40'): (1, 1)}
Times executed: 
1


References:

1) Improving Man-Optimal Stable Matchings by Minimum Change
of Preference Lists by Takao Inoshita, Robert W. Irving, Kazuo Iwama, Shuichi Miyazaki,and Takashi Nagase.

2) https://stackoverflow.com/questions/36381702/worst-case-for-stable-matchings